# STAR/dimensional schema queries

* Retrieve the number of memes per year in ascending order, as well as the oldest memes acrosstime categories ("added" and "year")

* Find the top 10 social network origins (excluding ‘unknown’ origin and normalizing terms).

* Which memes have the largest number of common tags?

* Data augmentation. Given a meme, find the most similar memes in the data set based on 
    1)similaritybasedonDandelionAPI cosine similarity using the "description" of each meme asinput and 
    2) the number of common tags.
    
* Data augmentation. Is it possible to find two memes that have their description similarityscore above 90% but have few tags in common tags?

* Data augmentation. Using the Google Vision data enrichment available data, get the distri-bution of safeness categories (adult, spoof, medical, violence and racy) as percentage of allthe memes analyzed.


In [9]:
#!conda install -c anaconda psycopg2
#import sys
#!conda install --yes --prefix {sys.prefix} psycopg2

In [63]:
import pandas as pd
import psycopg2
from psycopg2 import Error

connection = None
try:
    # Connect to an existing database
    connection = psycopg2.connect(user="airflow",
                                  password="airflow",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="airflow")

    # Create a cursor to perform database operations
    cursor = connection.cursor()
    # Print PostgreSQL details
    print("PostgreSQL server information")
    print(connection.get_dsn_parameters(), "\n")
    # Executing a SQL query
    cursor.execute("SELECT version();")
    # Fetch result
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)


PostgreSQL server information
{'user': 'airflow', 'dbname': 'airflow', 'host': '127.0.0.1', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 13.4 (Debian 13.4-1.pgdg110+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit',) 



##  Retrieve the number of memes per year in ascending order, as well as the oldest memes acrosstime categories ("added" and "year")



### Memes per year in ascending order

In [68]:
connection.rollback()
query = """
SELECT count(meme_kym_id) as total, year  from memes_dim group by year order by total desc, year ASC LIMIT 10;
"""
dat = pd.read_sql_query(query, connection)
dat.head(10)

,total,year
0,807,2011
1,756,2010
2,584,2016
3,522,2012
4,509,2014
5,500,0
6,490,2009
7,464,2015
8,449,2013
9,433,2017


### Oldest memes acrosstime categories "year"

In [80]:
connection.rollback()
query="""
SELECT 
    year, 
    meme_kym_id
FROM memes_dim 
WHERE year>0
ORDER BY year 
ASC LIMIT 10;
"""
dat = pd.read_sql_query(query, connection)
dat.head(10)


,year,meme_kym_id
0,1336,baka
1,1392,katana
2,1495,the-last-supper-parodies
3,1503,mona-lisa
4,1561,shits-on-fire-yo
5,1590,roses-are-red-violets-are-blue
6,1600,tarot-arcana-parodies
7,1605,guy-fawkes-mask
8,1700,breaking-the-fourth-wall
9,1700,bamboozle


### Oldest memes acrosstime categories "added"

In [81]:
connection.rollback()
query="""
SELECT 
    cast(to_timestamp(added) as date) AS date_added, 
    meme_kym_id 
FROM memes_dim 
WHERE added > 0
ORDER BY added ASC LIMIT 10;
"""
dat = pd.read_sql_query(query, connection)
dat.head(10)


,date_added,meme_kym_id
0,2008-12-10,rickroll
1,2008-12-10,technoviking
2,2008-12-10,miss-teen-usa-south-carolina
3,2008-12-10,chocolate-rain
4,2008-12-10,crank-that-soulja-boy
5,2008-12-12,all-your-base-are-belong-to-us
6,2008-12-12,star-wars-kid
7,2008-12-12,o-rly
8,2008-12-12,leave-britney-alone
9,2008-12-12,i-like-turtles


##  Find the top 10 social network origins (excluding ‘unknown’ origin and normalizing terms).

In [82]:
connection.rollback()
query="""
SELECT count(meme_kym_id) as total, origin  
FROM memes_dim 
WHERE origin <> 'Unknown'
GROUP BY origin order by total desc, origin 
LIMIT 10;
"""
dat = pd.read_sql_query(query, connection)
dat.head(10)


,total,origin
0,877,YouTube
1,593,Twitter
2,540,4chan
3,399,Tumblr
4,357,Reddit
5,198,niconico
6,163,Facebook
7,94,YTMND
8,76,Instagram
9,72,Vine



##   Which memes have the largest number of common tags?


In [90]:
connection.rollback()
query="""
SELECT 
    msf.common_tags,
    msf.tags_similarity,
	m1.meme_kym_id as meme1Id,
	m2.meme_kym_id as meme2Id,
    m1.tags as tags1,
	m2.tags as tags2

FROM memes_similarity_facts msf
INNER JOIN memes_dim m1 ON m1.meme_kym_id=msf.meme_id1
INNER JOIN memes_dim m2 ON m2.meme_kym_id=msf.meme_id2

ORDER BY msf.common_tags desc, tags_similarity desc
LIMIT 10
"""
dat = pd.read_sql_query(query, connection)
dat.head(10)


,common_tags,tags_similarity,meme1id,meme2id,tags1,tags2
0,9,1.00,im-so-happy,sakura-reflection,"[a midsummer night's lewd dream, bemani, japan...","[a midsummer night's lewd dream, bemani, japan..."
1,9,1.00,sakura-reflection,im-so-happy,"[a midsummer night's lewd dream, bemani, japan...","[a midsummer night's lewd dream, bemani, japan..."
2,9,0.75,kirby-sky-high-remixes,kirbys-gourmet-race-remixes,"[game music, japan, kirby, kirby super star, m...","[game music, japan, kirby, kirby super star, m..."
3,9,0.75,kirbys-gourmet-race-remixes,kirby-sky-high-remixes,"[game music, japan, kirby, kirby super star, m...","[game music, japan, kirby, kirby super star, m..."
4,9,0.68,oh-man-oh-god,theyre-eating-her,"[b movie, catchphrase, film, movie, movie quot...","[b movie, film, movie, nostalgia, pop culture ..."
5,9,0.68,theyre-eating-her,oh-man-oh-god,"[b movie, film, movie, nostalgia, pop culture ...","[b movie, catchphrase, film, movie, movie quot..."
6,8,0.96,clash-on-the-big-bridge,decisive-battle,"[cover, final fantasy, japan, mad video, nicon...","[cover, final fantasy, japan, mad video, nicon..."
7,8,0.96,decisive-battle,clash-on-the-big-bridge,"[cover, final fantasy, japan, mad video, nicon...","[cover, final fantasy, japan, mad video, nicon..."
8,8,0.88,dancing-mad,decisive-battle,"[cover, dancing mad, final fantasy, japan, mad...","[cover, final fantasy, japan, mad video, nicon..."
9,8,0.88,decisive-battle,dancing-mad,"[cover, final fantasy, japan, mad video, nicon...","[cover, dancing mad, final fantasy, japan, mad..."


##   Data augmentation. Given a meme, find the most similar memes in the data set based on 

### 1) similarity based on cosine similarity using the "description" of each meme as input and

In [91]:
connection.rollback()
query="""
select 
	msf.desc_similarity,
	msf.common_tags,
	msf.tags_similarity,
	m1.meme_kym_id as meme1Id,
	m2.meme_kym_id as meme2Id,
	m1.description as meme1Description,
	m2.description as meme2Description,
	m1.tags as meme1Tags,
	m2.tags as meme1Tags

from memes_similarity_facts msf
inner join memes_dim m1 ON m1.meme_kym_id=msf.meme_id1
inner join memes_dim m2 ON m2.meme_kym_id=msf.meme_id2

where msf.common_tags = 0 and msf.desc_similarity > 0.5
order by msf.desc_similarity desc
LIMIT 10
"""
dat = pd.read_sql_query(query, connection)
dat.head(10)

,desc_similarity,common_tags,tags_similarity,meme1id,meme2id,meme1description,meme2description,meme1tags,meme1tags
0,1.00,0,0.0,princess-celestia,twilight-sparkle,Use the main My Little Pony: Friendship is Mag...,Use the main My Little Pony: Friendship is Mag...,"[celestia, character, mlp, molestia, my little...","[alicorn, fim, friendship is magic, mlp charac..."
1,1.00,0,0.0,twilight-sparkle,princess-celestia,Use the main My Little Pony: Friendship is Mag...,Use the main My Little Pony: Friendship is Mag...,"[alicorn, fim, friendship is magic, mlp charac...","[celestia, character, mlp, molestia, my little..."
2,1.00,0,0.0,princess-celestia,rarity,Use the main My Little Pony: Friendship is Mag...,Use the main My Little Pony: Friendship is Mag...,"[celestia, character, mlp, molestia, my little...","[derpibooru, friendship is magic, gibbontake, ..."
3,1.00,0,0.0,rarity,princess-celestia,Use the main My Little Pony: Friendship is Mag...,Use the main My Little Pony: Friendship is Mag...,"[derpibooru, friendship is magic, gibbontake, ...","[celestia, character, mlp, molestia, my little..."
4,0.84,0,0.0,expectation-vs-reality,name-brand-vs-great-value,Expectation Vs. Reality refers to a series of ...,Name Brand vs. Great Value refers to a series ...,"[borders, burger king, comparison, fail, nigah...","[beyonce, beyoncé, black twitter, comparisons,..."
5,0.84,0,0.0,name-brand-vs-great-value,expectation-vs-reality,Name Brand vs. Great Value refers to a series ...,Expectation Vs. Reality refers to a series of ...,"[beyonce, beyoncé, black twitter, comparisons,...","[borders, burger king, comparison, fail, nigah..."
6,0.81,0,0.0,applejack,princess-celestia,Please use the My Little Pony : Friendship is ...,Use the main My Little Pony: Friendship is Mag...,"[applejack, friendship is magic, mlp character...","[celestia, character, mlp, molestia, my little..."
7,0.81,0,0.0,princess-celestia,applejack,Use the main My Little Pony: Friendship is Mag...,Please use the My Little Pony : Friendship is ...,"[celestia, character, mlp, molestia, my little...","[applejack, friendship is magic, mlp character..."
8,0.80,0,0.0,were-going-to-read-the-little-dolphin-who-cried,hey-plankton-can-our-first-song-go-like-this,The Little Dolphin Who Cried is a scene that h...,Hey Plankton! Can our first song go like this?...,"[dolphin, fairly odd parents, youtube poop, ytp]","[hey, plankton, spongebob, squarepants]"
9,0.80,0,0.0,buster-sword,interstellar-docking-parodies,[Warning: This entry contains major spoilers!],(Note: This Entry Contains Major Spoilers!),"[cloud, final fantasy, final fantasy vii, jrpg...","[docking, fan, interstellar, parody, satellite]"


### 2) the number of common tags.

##   Data augmentation. Is it possible to find two memes that have their description similarityscore above 90% but have few tags in common tags?

In [93]:
connection.rollback()
query="""
select 
	msf.desc_similarity,
	msf.common_tags,
	msf.tags_similarity,
	m1.meme_kym_id as meme1Id,
	m2.meme_kym_id as meme2Id,
	m1.description as meme1Description,
	m2.description as meme2Description,
	m1.tags as meme1Tags,
	m2.tags as meme1Tags

from memes_similarity_facts msf
inner join memes_dim m1 ON m1.meme_kym_id=msf.meme_id1
inner join memes_dim m2 ON m2.meme_kym_id=msf.meme_id2

where 
    msf.common_tags = 0 AND msf.desc_similarity > 0.9
order by msf.desc_similarity desc
LIMIT 10
"""
dat = pd.read_sql_query(query, connection)
dat.head(10)

,desc_similarity,common_tags,tags_similarity,meme1id,meme2id,meme1description,meme2description,meme1tags,meme1tags
0,1.0,0,0.0,rarity,princess-celestia,Use the main My Little Pony: Friendship is Mag...,Use the main My Little Pony: Friendship is Mag...,"[derpibooru, friendship is magic, gibbontake, ...","[celestia, character, mlp, molestia, my little..."
1,1.0,0,0.0,twilight-sparkle,princess-celestia,Use the main My Little Pony: Friendship is Mag...,Use the main My Little Pony: Friendship is Mag...,"[alicorn, fim, friendship is magic, mlp charac...","[celestia, character, mlp, molestia, my little..."
2,1.0,0,0.0,princess-celestia,rarity,Use the main My Little Pony: Friendship is Mag...,Use the main My Little Pony: Friendship is Mag...,"[celestia, character, mlp, molestia, my little...","[derpibooru, friendship is magic, gibbontake, ..."
3,1.0,0,0.0,princess-celestia,twilight-sparkle,Use the main My Little Pony: Friendship is Mag...,Use the main My Little Pony: Friendship is Mag...,"[celestia, character, mlp, molestia, my little...","[alicorn, fim, friendship is magic, mlp charac..."


* Data augmentation. Using the Google Vision data enrichment available data, get the distri-bution of safeness categories (adult, spoof, medical, violence and racy) as percentage of allthe memes analyzed.